In [115]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import minimize
import matplotlib.gridspec as gridspec
from datetime import date, timedelta
import geopandas as gpd

#import today date
date_today = date.today()
year_t,month_t,date_t=str(date_today).split('-')

# The SIR model differential equations.
def deriv(y, t, N, beta,gamma):
    S,I,R = y

    dSdt = -(beta*I/N)*S 
    dIdt = (beta*S/N)*I - gamma*I 
    dRdt = gamma*I 
    
    return dSdt, dIdt, dRdt

#Integration of the differential equations
    
def time_evo(N,beta,gamma,I0=1,R0=0,t=np.arange(0,365)):
    # Definition of the initial conditions
    # I0 and R0 denotes the number of initial infected people (I0) 
    # and the number of people that recovered and are immunized (R0)
    
    # t ise the timegrid
    
    S0=N-I0-R0  # number of people that can still contract the virus
    
    # Initial conditions vector
    y0 = S0, I0, R0

    # Integrate the SIR equations over the time grid, t.
    ret = odeint(deriv, y0, t, args=(N,beta,gamma))
    S, I, R = np.transpose(ret)
    
    return (t,S,I,R)

countries_list=['Albania',
                'Armenia',
                'Austria',
                'Azerbaijan',
                'Belarus',
                'Belgium',
                'Bosnia and Herzegovina',
                'Bulgaria',
                'Cyprus',
                'Croatia',
                'Czechia',
                'Denmark',
                'Estonia',
                'Finland',
                'France',
                'Georgia',
                'Germany',
                'Greece',
                'Hungary',
                'Iceland',
                'Ireland',
                'Israel',
                'Italy',
                'Kazakhstan',
                'Kyrgyzstan',
                'Latvia',
                'Lithuania',
                'Luxembourg',
                'Malta',
                'Moldova',
                'Monaco',
                'Montenegro',
                'Netherlands',
                'North Macedonia',
                'Norway',
                'Poland',
                'Portugal',
                'Romania',
                'Serbia',
                'Slovakia',
                'Slovenia',
                'Spain',
                'Sweden',
                'Switzerland',
                'Turkey',
                'Ukraine',
                'United Kingdom']

#IMPORT FILES WORLD
#i files sono: le righe sono le nazioni, le colonne i giorni del mese (DATE).

file_confirmed='https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
file_deaths='https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
file_recovered='https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv'

df_confirmed=pd.read_csv(file_confirmed)
df_deaths=pd.read_csv(file_deaths)
df_recovered=pd.read_csv(file_recovered)

countries_w_confirmed = df_confirmed['Country/Region']
countries_w_deaths = df_deaths['Country/Region']
countries_w_recovered = df_recovered['Country/Region']

#confirmed world
confirmed_world0 = df_confirmed.drop(['Province/State','Lat','Long'], 
                                    axis=1)
confirmed_world0.rename(index=countries_w_confirmed, inplace=True)
confirmed_world = confirmed_world0.drop(['Country/Region'], 
                                        axis=1).T.reset_index()
confirmed_world.rename(columns={'index':'Date'}, inplace=True)

#deaths world
deaths_world0 = df_deaths.drop(['Province/State','Lat','Long'], 
                                    axis=1)
deaths_world0.rename(index=countries_w_deaths, inplace=True)
deaths_world = deaths_world0.drop(['Country/Region'], 
                                        axis=1).T.reset_index()
deaths_world.rename(columns={'index':'Date'}, inplace=True)

#recovered world
recovered_world0 = df_recovered.drop(['Province/State','Lat','Long'], 
                                    axis=1)
recovered_world0.rename(index=countries_w_recovered, inplace=True)
recovered_world = recovered_world0.drop(['Country/Region'], 
                                        axis=1).T.reset_index()
recovered_world.rename(columns={'index':'Date'}, inplace=True)

confirmed_europe0 = confirmed_world[countries_list]
deaths_europe0 = deaths_world[countries_list]
recovered_europe0 = recovered_world[countries_list]

array_names=([])
for name in countries_list:
    array_names.append([name,list(countries_w_confirmed).count(name)])

Totale=pd.DataFrame()
for i in range(0, len(countries_list)):
    if array_names[i][1] > 1:
               Totale.insert(i, 
                              countries_list[i], 
                              value=confirmed_europe0[countries_list[i]].T.sum())
    elif array_names[i][1]==1:
                Totale.insert(i, 
                                countries_list[i], 
                                value=confirmed_europe0[countries_list[i]].T)

Totale.insert(0, 'Date', confirmed_world['Date'])

Deceduti=pd.DataFrame()
for i in range(0, len(countries_list)):
    if array_names[i][1] > 1:
            Deceduti.insert(i, 
                              countries_list[i], 
                              value=deaths_europe0[countries_list[i]].T.sum())
    elif array_names[i][1]==1:
            Deceduti.insert(i, 
                              countries_list[i], 
                              value=deaths_europe0[countries_list[i]].T)

Deceduti.insert(0, 'Date', deaths_world['Date'])

Guariti=pd.DataFrame()
for i in range(0, len(countries_list)):
    if array_names[i][1] > 1:
                Guariti.insert(i, 
                              countries_list[i], 
                              value=recovered_europe0[countries_list[i]].T.sum())
    elif array_names[i][1]==1:
                Guariti.insert(i, 
                                 countries_list[i], 
                                 value=recovered_europe0[countries_list[i]].T)

Guariti.insert(0, 'Date', recovered_world['Date'])

#Active Infected
Attualmente_positivi=pd.DataFrame()

for i in range(0, len(countries_list)):
    Attualmente_positivi.insert(i, 
                                  countries_list[i], 
                                  value=
                                     Totale[countries_list[i]]-
                                      Deceduti[countries_list[i]]-
                                      Guariti[countries_list[i]])

Attualmente_positivi.insert(0, 'Date', confirmed_world['Date'])

Totale.to_csv('output/10_tot_casi_europe_'+date_t+month_t+'.csv', index=True)
Deceduti.to_csv('output/10_deceduti_europe_'+date_t+month_t+'.csv', index=True)
Guariti.to_csv('output/10_guariti_europe_'+date_t+month_t+'.csv', index=True)
Attualmente_positivi.to_csv('output/10_attualmente_positivi_europe_'+date_t+month_t+'.csv', index=True)

#Daily variation infected
Variazione_giornaliera = pd.DataFrame(Attualmente_positivi['Date'].iloc[1:])

for name in countries_list:
    active_var=([])    
    for i in range(1,len(Attualmente_positivi)):
        active_var.append(Attualmente_positivi[name][i]-Attualmente_positivi[name][i-1])
    Variazione_giornaliera[name]=active_var

Variazione_giornaliera.to_csv('output/10_variazione_giornaliera_europe_'+date_t+month_t+'.csv', index=True)

def func_plot(df):
    
    y_world=[]
    n_cols=df.shape[1]
    
    for i in range(n_cols-4):
        y_world.append(df.iloc[:,i+4].sum())
    
    x_world2=df.columns[4:]
    x_world=pd.to_datetime(x_world2,infer_datetime_format=False)
    
    return (x_world,y_world)

#Generalization to other countries

def whichcountry(name):

    ######## INPUT PARAMETERS ########
    country=name
    t0=pd.to_datetime('2020-01-22')
    #################################

    mask_coun=df_confirmed['Country/Region']==country   # you can change the country here
    mask_coun_rec=df_recovered['Country/Region']==country

    df_confirmed_C=df_confirmed.loc[mask_coun,:]
    df_deaths_C=df_deaths.loc[mask_coun,:]
    df_recovered_C=df_recovered.loc[mask_coun_rec,:]

    ytot=np.array(func_plot(df_confirmed_C)[1])
    ydeaths=np.array(func_plot(df_deaths_C)[1])
    yrec=np.array(func_plot(df_recovered_C)[1])

    return ytot-ydeaths-yrec, ytot[-1], yrec[-1],ydeaths[-1]

xdata=pd.to_numeric(range(Attualmente_positivi.shape[0]))

today=len(xdata)

def minimizer(R0,t1=today-5,t2=today):
    array_country_bis=array_country
    
    #true data
    ydata_inf_2=array_country[t1:t2]
    xdata_2=np.arange(0,len(ydata_inf_2))
    
    #model
    fin_result=time_evo(60*10**6,1/14*R0,1/14,I0=ydata_inf_2[0])
    i_vec=fin_result[2]
    i_vec_2=i_vec[0:len(xdata_2)]
    
    #average error
    error=np.sum(np.abs(ydata_inf_2-i_vec_2)/ydata_inf_2)*100
    return error

minimizer_vec=np.vectorize(minimizer)

time_window=5

def minimizer_gen(t1,t2,xgrid=np.arange(0.1,5,0.01)):

    ygrid=minimizer_vec(xgrid,t1=t1,t2=t2)
    r0_ideal=round(xgrid[np.argmin(ygrid)],2)

    return r0_ideal

r0_today=[]
scangrid=np.linspace(0,3,400)
name_R0_array = []

for name in range(0, len(countries_list)):
    
    array_country=whichcountry(countries_list[name])[0]
    
    i = today-(time_window-1)
    min_today=minimizer_gen(i,i+time_window,scangrid)
    r0_today.append(min_today)
    #scangrid=np.linspace(0,5,200)
    name_R0_array.append([countries_list[name], min_today])

name_R0_df = pd.DataFrame(name_R0_array, columns=['Country', 'R0'])

countries_hist=['United Kingdom',
                'Ukraine',
                'Poland',
                'Greece',
                'Netherlands',
                'Portugal',
                'Belgium',
                'France',
                'Slovenia',
                'Serbia',
                'Spain',
                'Italy',
                'Sweden',
                'Austria',
                'Slovakia',
                'Turkey']

hist_list=[]
for i in range(len(countries_hist)):
    ind = name_R0_df.loc[name_R0_df['Country'] == countries_hist[i]].index[0]
    hist_list.append([name_R0_df['Country'][ind], name_R0_df['R0'][ind]])
hist_df = pd.DataFrame(hist_list, columns=['Country', 'R0'])
hist_df.to_csv('output/10_R0_europe_hist_'+date_t+month_t+'.csv')

##import yesterday date
#yesterday = date.today() - timedelta(days=1)
#year_y,month_y,date_y=str(yesterday).split('-')
#
#r0_countries_imp = pd.read_excel('input/input.xlsx')
#
#
#row_today=pd.DataFrame(np.reshape(r0_today,(1, len(countries_list))),
#                       index= [str(yesterday)],
#                       columns=countries_list).reset_index()
#row_today.rename(columns={'index':'Data'}, inplace=True)
#row_today.index = [len(r0_countries_imp)]
#
#export_today = pd.concat([r0_imp_noindex,row_today])
#export_today.to_excel('output/10_R0_europe_curve_'+date_t+month_t+'.xlsx',index=True)
#export_today.to_excel('input/input.xlsx', index = True)
#
#r0_to_join = pd.Series(name_R0_df['R0'])
#r0_to_join.index = name_R0_df['Country']
#confirmed_to_join = Totale.iloc[-1, 1:]
#deaths_to_join = Deceduti.iloc[-1, 1:]
#recovered_to_join = Guariti.iloc[-1, 1:]
#ai_to_join = Attualmente_positivi.iloc[-1, 1:]
#
#frame = {'R0':r0_to_join,
#         'Confirmed': confirmed_to_join, 
#         'Deaths': deaths_to_join, 
#         'Recovered':recovered_to_join, 
#         'Active Infected': ai_to_join}
#
#df_to_join = pd.DataFrame(frame)
#df_to_join.rename(index={'Czechia':'Czech Republic', 
#                         'Moldova':'Republic of Moldova', 
#                         'North Macedonia':'The former Yugoslav Republic of Macedonia'})
#df_to_join.reset_index()
#
##Map
#map = gpd.read_file("https://raw.githubusercontent.com/leakyMirror/map-of-europe/master/GeoJSON/europe.geojson")
#map = map.join(df_to_join, on='NAME', how='left')

/Users/luigigiuseppeatzeni/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:259: RuntimeWarning: invalid value encountered in true_divide


In [116]:
#import yesterday date
yesterday = date.today() - timedelta(days=1)
year_y,month_y,date_y=str(yesterday).split('-')

r0_countries_imp = pd.read_excel('input/input.xlsx')


In [117]:
r0_countries_imp.tail()

,Data,Albania,Armenia,Austria,Azerbaijan,Belarus,Belgium,Bosnia and Herzegovina,Bulgaria,Cyprus,...,Romania,Serbia,Slovakia,Slovenia,Spain,Sweden,Switzerland,Turkey,Ukraine,United Kingdom
91,2020-06-10,1.54,1.30,0.93,1.59,1.14,1.03,1.49,1.43,1.39,...,0.93,0.98,0.16,1.57,1.04,1.23,1.30,0.00,1.21,1.07
92,2020-06-11,1.84,1.01,0.83,1.52,1.08,1.03,1.63,1.62,1.32,...,0.98,4.03,0.57,1.55,1.06,1.22,1.20,0.00,1.22,1.07
93,2020-06-12,1.96,0.97,0.50,1.57,0.92,1.03,1.43,1.70,1.21,...,1.11,1.32,0.92,1.91,1.07,1.34,0.75,0.28,1.19,1.07
94,2020-06-13,1.85,1.20,0.66,1.62,0.80,1.03,1.68,1.78,1.19,...,1.11,1.62,1.19,1.94,1.10,1.40,0.61,0.69,1.21,1.07
95,2020-06-14,1.82,1.56,0.63,1.50,0.79,1.03,1.48,1.77,1.21,...,1.17,1.96,1.28,1.85,1.10,1.42,0.76,1.09,1.28,1.07


In [123]:
row_today=pd.DataFrame(np.reshape(r0_today,(1, len(countries_list))),
                       index= [str(yesterday)],
                       columns=countries_list).reset_index()
row_today.rename(columns={'index':'Data'}, inplace=True)
row_today.index = [len(r0_countries_imp)]

In [124]:
row_today.head()

,Data,Albania,Armenia,Austria,Azerbaijan,Belarus,Belgium,Bosnia and Herzegovina,Bulgaria,Cyprus,...,Romania,Serbia,Slovakia,Slovenia,Spain,Sweden,Switzerland,Turkey,Ukraine,United Kingdom
96,2020-06-15,2.7,1.53,0.47,1.53,0.74,1.02,1.0,1.28,1.14,...,1.34,1.35,1.33,2.13,1.07,1.27,0.5,1.28,1.4,1.08


In [125]:
export_today = pd.concat([r0_countries_imp,row_today])

In [126]:
export_today.tail(10)

,Data,Albania,Armenia,Austria,Azerbaijan,Belarus,Belgium,Bosnia and Herzegovina,Bulgaria,Cyprus,...,Romania,Serbia,Slovakia,Slovenia,Spain,Sweden,Switzerland,Turkey,Ukraine,United Kingdom
87,2020-06-06 00:00:00,1.54,2.03,0.66,2.10,0.94,0.99,1.08,0.19,0.76,...,0.90,0.00,0.96,2.49,1.07,1.53,0.00,0.89,1.09,1.10
88,2020-06-07 00:00:00,1.35,1.94,1.05,2.00,0.96,1.02,1.22,0.21,0.49,...,0.98,0.00,0.81,2.86,1.07,1.38,0.73,0.74,1.10,1.07
89,2020-06-08 00:00:00,1.18,1.74,1.31,1.93,1.02,1.03,0.97,0.58,0.61,...,0.81,0.00,0.55,3.79,1.07,1.31,0.40,0.43,1.20,1.07
90,2020-06-09 00:00:00,1.24,1.49,1.14,1.74,1.12,1.03,1.29,0.93,1.27,...,0.79,0.00,0.45,3.00,1.07,1.27,0.19,0.11,1.18,1.07
91,2020-06-10 00:00:00,1.54,1.30,0.93,1.59,1.14,1.03,1.49,1.43,1.39,...,0.93,0.98,0.16,1.57,1.04,1.23,1.30,0.00,1.21,1.07
92,2020-06-11 00:00:00,1.84,1.01,0.83,1.52,1.08,1.03,1.63,1.62,1.32,...,0.98,4.03,0.57,1.55,1.06,1.22,1.20,0.00,1.22,1.07
93,2020-06-12 00:00:00,1.96,0.97,0.50,1.57,0.92,1.03,1.43,1.70,1.21,...,1.11,1.32,0.92,1.91,1.07,1.34,0.75,0.28,1.19,1.07
94,2020-06-13 00:00:00,1.85,1.20,0.66,1.62,0.80,1.03,1.68,1.78,1.19,...,1.11,1.62,1.19,1.94,1.10,1.40,0.61,0.69,1.21,1.07
95,2020-06-14 00:00:00,1.82,1.56,0.63,1.50,0.79,1.03,1.48,1.77,1.21,...,1.17,1.96,1.28,1.85,1.10,1.42,0.76,1.09,1.28,1.07
96,2020-06-15,2.70,1.53,0.47,1.53,0.74,1.02,1.00,1.28,1.14,...,1.34,1.35,1.33,2.13,1.07,1.27,0.50,1.28,1.40,1.08


In [127]:
export_today.to_excel('output/10_R0_europe_curve_'+date_t+month_t+'.xlsx',index=True)
export_today.to_excel('input/input.xlsx', index = True)

In [128]:
r0_to_join = pd.Series(name_R0_df['R0'])
r0_to_join.index = name_R0_df['Country']

In [129]:
r0_to_join.head()

Country
Albania       2.70
Armenia       1.53
Austria       0.47
Azerbaijan    1.53
Belarus       0.74
Name: R0, dtype: float64

In [130]:
confirmed_to_join = Totale.iloc[-1, 1:]
deaths_to_join = Deceduti.iloc[-1, 1:]
recovered_to_join = Guariti.iloc[-1, 1:]
ai_to_join = Attualmente_positivi.iloc[-1, 1:]

frame = {'R0':r0_to_join,
         'Confirmed': confirmed_to_join, 
         'Deaths': deaths_to_join, 
         'Recovered':recovered_to_join, 
         'Active Infected': ai_to_join}

In [131]:
df_to_join = pd.DataFrame(frame)
df_to_join.rename(index={'Czechia':'Czech Republic', 
                         'Moldova':'Republic of Moldova', 
                         'North Macedonia':'The former Yugoslav Republic of Macedonia'})

,R0,Confirmed,Deaths,Recovered,Active Infected
Country,,,,,
Albania,2.70,1590,36,1055,499
Armenia,1.53,17064,285,6276,10503
Austria,0.47,17135,678,16066,391
Azerbaijan,1.53,10324,122,5739,4463
Belarus,0.74,54680,312,30420,23948
Belgium,1.02,60100,9661,16610,33829
Bosnia and Herzegovina,1.00,3040,165,2162,713
Bulgaria,1.28,3341,176,1784,1381
Cyprus,1.14,985,18,807,160


In [132]:
df_to_join.reset_index()

,Country,R0,Confirmed,Deaths,Recovered,Active Infected
0,Albania,2.70,1590,36,1055,499
1,Armenia,1.53,17064,285,6276,10503
2,Austria,0.47,17135,678,16066,391
3,Azerbaijan,1.53,10324,122,5739,4463
4,Belarus,0.74,54680,312,30420,23948
5,Belgium,1.02,60100,9661,16610,33829
6,Bosnia and Herzegovina,1.00,3040,165,2162,713
7,Bulgaria,1.28,3341,176,1784,1381
8,Cyprus,1.14,985,18,807,160
9,Croatia,0.00,2254,107,2140,7


In [133]:
#Map
map = gpd.read_file("https://raw.githubusercontent.com/leakyMirror/map-of-europe/master/GeoJSON/europe.geojson")
map = map.join(df_to_join, on='NAME', how='left')

In [114]:
map


,FID,FIPS,ISO2,ISO3,UN,NAME,AREA,POP2005,REGION,SUBREGION,LON,LAT,geometry,R0,Confirmed,Deaths,Recovered,Active Infected
0,0.0,AJ,AZ,AZE,31,Azerbaijan,8260,8352021,142,145,47.395,40.430,"(POLYGON ((45.083321 39.76804400000015, 45.266...",1.53,10324,122,5739,4463
1,0.0,AL,AL,ALB,8,Albania,2740,3153731,150,39,20.068,41.143,"POLYGON ((19.436214 41.021065, 19.450554 41.05...",2.70,1590,36,1055,499
2,0.0,AM,AM,ARM,51,Armenia,2820,3017661,142,145,44.563,40.534,"(POLYGON ((45.57305100000013 40.632488, 45.528...",1.53,17064,285,6276,10503
3,0.0,BK,BA,BIH,70,Bosnia and Herzegovina,5120,3915238,150,39,17.786,44.169,"POLYGON ((17.649841 42.889076, 17.578526 42.94...",1.00,3040,165,2162,713
4,0.0,BU,BG,BGR,100,Bulgaria,11063,7744591,150,151,25.231,42.761,"POLYGON ((27.879166 42.841103, 27.894997 42.80...",1.28,3341,176,1784,1381
5,0.0,CY,CY,CYP,196,Cyprus,924,836321,142,145,33.219,35.043,"POLYGON ((33.652618 35.354103, 33.713051 35.38...",1.14,985,18,807,160
6,0.0,DA,DK,DNK,208,Denmark,4243,5416945,150,154,9.264,56.058,"(POLYGON ((11.513887 54.82972000000012, 11.564...",1.15,12417,598,11290,529
7,0.0,EI,IE,IRL,372,Ireland,6889,4143294,150,154,-8.152,53.177,"(POLYGON ((-9.65639 53.222221, -9.663334000000...",1.38,25321,1706,22698,917
8,0.0,EN,EE,EST,233,Estonia,4239,1344312,150,154,25.793,58.674,"(POLYGON ((23.990829 58.099998, 23.97805000000...",0.76,1974,69,1717,188
9,0.0,AU,AT,AUT,40,Austria,8245,8291979,150,155,14.912,47.683,"POLYGON ((13.833611 48.773605, 13.858055 48.77...",0.47,17135,678,16066,391


In [137]:
map.to_file('output/10_mappa_R0_europa_'+date_t+month_t+'.geojson', driver='GeoJSON')

GeometryTypeValidationError: Record's geometry type does not match collection schema's geometry type: 'MultiPolygon' != 'Polygon'